In [1]:
from pyspark import *
sc = SparkContext('local[*]')
sqlContext = 

<h1>Yelp Data Exploration</h1>

Remove logs

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import seaborn as sns

import datetime
import matplotlib.dates as mdates
import matplotlib.patches as mpatches
from pyspark.sql.functions import *

from pyspark.sql import *
%matplotlib inline

In [3]:
# remove spark heavy logging
log4j = sc._jvm.org.apache.log4j
log4j.LogManager.getRootLogger().setLevel(log4j.Level.ERROR)

<h1>Import Data</h1>

In [10]:
# HDFS Base path for Yelp Data
base_path = '/home/jovyan/work/yelp/json'

In [11]:
# Yelp file types list
file_type_list = ['businesses', 'checkins', 'reviews', 'tips', 'users']


In [19]:

# Dataframe dictionnary for each file type
dataframe_map = {t: sqlContext.read.json(base_path + '/%s' % t)
                   for t in file_type_list}


In [20]:
%timeit print(dataframe_map['users'].count())

552339
552339
552339
552339
1 loop, best of 3: 2.27 s per loop


In [ ]:
 print(dataframe_map['reviews'].count())

In [ ]:
df_users = dataframe_map['users']
df_reviews = dataframe_map['reviews']
df_business = dataframe_map['businesses']

<h1>Data Analysis</h1>
We gonna try to spot one <b>Yelper Star</b> and see what are the implications of his reviews

<h2>Friends Oriented</h2>

Find the user with the most friends 

In [ ]:
#Count the number of friends for everyuser

friends_df = df_users.select(df_users.user_id, size(df_users.friends))
oldnames = friends_df.schema.names

friends_df = friends_df.withColumnRenamed('size(friends)', 'friends_count')
#friends_df.printSchema()

#Sort the data descending
friends_df = friends_df.sort(friends_df.friends_count.desc())

yelper_star_id = friends_df.head(1)[0]['user_id']
print(yelper_star_id)
#Describe the Yelper star found

df_users.select('user_id', 'name', 'average_stars', 'review_count', 'fans', size(df_users.friends)).where(df_users.user_id == yelper_star_id).show()

#yelper_star.show()

The goal here is that for every reviews made by <b>Philip</b> on a business, we see how many reviews followed in the next month

In [ ]:
yelper_star_reviews  = df_reviews.where(df_reviews.user_id == yelper_star_id)

def strToDate(row) :
    newRow = Row(review_id=row.review_id, 
                 business_id=row.business_id, 
                 date=datetime.datetime.strptime(row.date, '%Y-%m-%d'))
    return newRow

yelper_star_reviews = yelper_star_reviews.map(lambda r : strToDate(r)).collect()


In [ ]:
#Turn date field of review into datetime object
timed_reviews = df_reviews.map(lambda r : strToDate(r))
timed_reviews = sqlContext.createDataFrame(timed_reviews)
timed_reviews.show()

In [ ]:
#Parse each of the yelper_star reviews
mini = yelper_star_reviews[:2]
print(len(mini))
for star_review in mini:
    start_date = star_review['date']
    
    
    end_date = start_date + datetime.timedelta(days=30)
    mini_reviews = timed_reviews.where((timed_reviews.business_id == star_review.business_id)
                                       &(timed_reviews.date > start_date) 
                                       & (timed_reviews.date <= end_date))
    mini_reviews = mini_reviews.groupBy('business_id').agg({'review_id':'count'})
    mini_reviews = mini_reviews.select('count(review_id)')
    
    print(mini_reviews.show())